# PkgEvalAnalysis

Latest pkgeval: https://s3.amazonaws.com/julialang-reports/nanosoldier/pkgeval/by_hash/64a9bb9_vs_2ca8b0c/report.html

In [1]:
using DataFrames, Feather

In [2]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    SyntaxError
    VisualRegression
    MissingDep
    BuildError
    UnsatReq
    BSONDataTypeField
    Belapsed
    StderrCheck
    
    # From PkgEval
    MissingDependency
    Inactivity
    MissingBinary
    Untestable
    LogLimit
    TimeLimit
    Syntax
    
    # Specific ones
    Cassette
    LLVMVersion
    UninstalledDep
    ImageReference
    IOBufferWritable
    JSONUndefinedReference
    CcallType
    NoMethodZero
    CloseWrite
    ReturnType
    StdinBuffer
    BlockArraysIsLess
    TypeNameConstField
    CxxWrap
    InfinityArrays
    
end


In [3]:
# Unpack the data unless it has already been unpacked

if !isdir("data")
    run(`tar -xvf data.tar.xz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

In [4]:
# Join the primary and against data

package_results = leftjoin(primary, against,
     on=:uuid, makeunique=true, indicator=:source);

In [5]:
# Filter out packages that started failing but didn't start failing when the RNG stream changed

fails = filter(test->test.source == "both" &&
                 test.status != test.status_1 &&
                 test.status in (":fail", ":kill"), package_results)

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

missing_binary_idx     = fails.reason .== ":binary_dependency"
inactivity_idx         = fails.reason .== ":inactivity"
missing_dependency_idx = fails.reason .== ":missing_dependency"
untestable_idx         = fails.reason .== ":untestable"
log_limit_idx          = fails.reason .== ":log_limit"
time_limit_idx         = fails.reason .== ":time_limit"

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

fails.why[missing_binary_idx]     .= MissingBinary
fails.why[inactivity_idx]         .= Inactivity
fails.why[missing_dependency_idx] .= MissingDependency
fails.why[untestable_idx]         .= Untestable
fails.why[log_limit_idx]          .= LogLimit
fails.why[time_limit_idx]         .= TimeLimit

nothing

In [6]:
# Some utility functions

query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [7]:
query(fails, "probe_platform_engines!")

,julia,compiled,name,uuid,version,status,reason,duration,log,julia_1
,String,Bool,String,String,String,String,String,Float64,String,String?


In [8]:
# Here we pattern match certain test errors and categorize them based on that.

# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "Error: doctest failure in ", DocTest)

update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, r"Expression: (\S*) ≈ (\S*)", ApproxError) 
update_reason!(fails, "Expression: ≈(", ApproxError)
    
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, " ambiguities found", NewAmbiguity)

update_reason!(fails, "Unsatisfiable requirements detected for package", UnsatReq)



update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
update_reason!(fails, r"Expression: (\N.*?) == ", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)
update_reason!(fails, "LoadError: syntax", SyntaxError)

update_reason!(fails, "Image did not match reference image", VisualRegression)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

update_reason!(fails, "Error building ", BuildError)
update_reason!(fails, "isempty(stderr_content)", StderrCheck)




# Specific ones for this release
update_reason!(fails, "Cassette.overdub", Cassette)
update_reason!(fails, "libLLVM-12jl.so", LLVMVersion)
update_reason!(fails, "You may have a partially installed environment", UninstalledDep)
update_reason!(fails, "- REFERENCE -------------------", ImageReference)
update_reason!(fails, "ArgumentError: ensureroom failed, IOBuffer is not writeable", IOBufferWritable)
update_reason!(fails, "JSON.Writer ~/.julia/packages/JSON/NeJ9k/src/Writer.jl:285", JSONUndefinedReference)
update_reason!(fails, "ccall method definition: argument 1 type doesn't correspond to a C type", CcallType)
update_reason!(fails, "no method matching zero", NoMethodZero)
update_reason!(fails, "closewrite not defined", CloseWrite)
update_reason!(fails, "no method matching return_type", ReturnType)
update_reason!(fails, "type IOStream has no field buffer", StdinBuffer)
update_reason!(fails, "UndefVarError: _isless not defined", BlockArraysIsLess)
update_reason!(fails, "setfield!: const field .names of type TypeName cannot be changed", TypeNameConstField)
update_reason!(fails, "C++ exception while wrapping module", CxxWrap)
update_reason!(fails, "no method matching Infinities.Infinity(::Float64)", InfinityArrays)






nothing

In [9]:
# Packages that have gotten  

issues_opened = [
    "StaticKernels" # https://github.com/stev47/StaticKernels.jl/issues/5
    "Polyhedra" # https://github.com/JuliaPolyhedra/Polyhedra.jl
    "TexasHoldEm" # https://github.com/charleskawczynski/TexasHoldem.jl/issues/106
    "RedefStructs" # https://github.com/FedericoStra/RedefStructs.jl/issues/4
    "Memento" # https://github.com/invenia/Memento.jl/issues/184
]

likely_tol = [
    "TaylorModels"
];

fixed = [
    "GeoDataFrames" # https://github.com/evetion/GeoDataFrames.jl/pull/30
]




ignored_packages = [
]

Any[]

In [10]:
# Total package failures that we haven't categorized

total_unknown(fails)

54

# Examples

Here are some examples of how one might do queries and categorize errors

In [11]:
# Unknown failures

filter(x -> x.why == Unknown && 
       !(x.name in issues_opened) && 
       !(x.name in likely_tol) && 
       !(x.name in ignored_packages) && 
       !(x.name in fixed), 
    fails)

,julia,compiled,name,uuid
,String,Bool,String,String
1,"v""1.8.0-DEV-8f13bff82b""",0,CodeInfoTools,"UUID(""bc773b8a-8374-437a-b9f2-0e9785855863"")"
2,"v""1.8.0-DEV-8f13bff82b""",0,GPUCompiler,"UUID(""61eb1bfa-7361-4325-ad38-22787b887f55"")"
3,"v""1.8.0-DEV-8f13bff82b""",0,Bonsai,"UUID(""30a8ad1e-2947-4f5c-9f0c-168e614e8e53"")"
4,"v""1.8.0-DEV-8f13bff82b""",0,GarishPrint,"UUID(""b0ab02a7-8576-43f7-aa76-eaa7c3897c54"")"
5,"v""1.8.0-DEV-8f13bff82b""",0,JET,"UUID(""c3a54625-cd67-489e-a8e7-0a5a0ff4e31b"")"
6,"v""1.8.0-DEV-8f13bff82b""",0,ArrayLayouts,"UUID(""4c555306-a7a7-4459-81d9-ec55ddd5c99a"")"
7,"v""1.8.0-DEV-8f13bff82b""",0,EquationsOfStateOfSolids,"UUID(""1eaa2786-f833-4167-8397-974edad0881e"")"
8,"v""1.8.0-DEV-8f13bff82b""",0,LiterateTest,"UUID(""d77d25b0-90d3-4a16-b10a-412a9d48f625"")"
9,"v""1.8.0-DEV-8f13bff82b""",0,TensorKitManifolds,"UUID(""11fa318c-39cb-4a83-b1ed-cdc7ba1e3684"")"


In [12]:
# Categorize

z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    n > 0 && push!(z, (n, i))
end
sort!(z; rev=true)

31-element Vector{Any}:
 (54, Unknown)
 (34, PrintingChange)
 (19, NewExceptionType)
 (17, CcallType)
 (16, UninstalledDep)
 (14, LLVMVersion)
 (7, SyntaxError)
 (6, CxxWrap)
 (6, ApproxError)
 (6, NewAmbiguity)
 (4, InfinityArrays)
 (4, ReturnType)
 (4, DocTest)
 ⋮
 (2, UnsatReq)
 (2, InferredFailure)
 (1, TypeNameConstField)
 (1, JSONUndefinedReference)
 (1, IOBufferWritable)
 (1, ImageReference)
 (1, Cassette)
 (1, Inactivity)
 (1, StderrCheck)
 (1, VisualRegression)
 (1, UnexpectedPass)
 (1, TestAmbiguities)

In [13]:
# Package names of some particular failure

d = filter(x -> x.why == PrintingChange, fails).name
#d = filter(!in(issues_opened), d)
#d = filter(!in(likely_tol), d)
#join(d, '\n') |> print

34-element Vector{String}:
 "LoweredCodeUtils"
 "ConjugatePriors"
 "LogRoller"
 "Signals"
 "AxisTables"
 "QuasiArrays"
 "StarAlgebras"
 "BlockArrays"
 "QuantumESPRESSOCommands"
 "DeepDiffs"
 "TimeSeries"
 "CBinding"
 "LegibleLambdas"
 ⋮
 "SnoopCompile"
 "FeatureEng"
 "ChainRulesCore"
 "SimplePadics"
 "FinRua"
 "StrRegex"
 "TraitSimulation"
 "HomotopyContinuation"
 "MIRT"
 "ControlSystems"
 "CompactBases"
 "PProf"